<a href="https://colab.research.google.com/github/Roghiehghahremani/Deep-Learning-With-PyTorch---Full-Course_Patrick-Loeber/blob/main/15_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# ImageFolder
# Scheduler
# Transfer learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets,models,transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/DLPyTorch_Patrick_Loeber

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/DLPyTorch_Patrick_Loeber


In [12]:
mean = np.array([0.485, 0.456,0.406])
std  = np.array([0.229, 0.224, 0.225])

data_transforms ={
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
}

In [19]:
# import data
data_dir = 'data/hymenoptera_data'
sets = ['train','val']

image_datasets = {x:datasets.ImageFolder(os.path.join(data_dir,x), data_transforms[x]) for x in ['train', 'val']}

data_loader  = {x:torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0) for x in ['train', 'val']}

dataset_sizes ={x: len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(class_names)

['ants', 'bees']
